## prerequisites:

- Beautifulsoup
- selenium
- pandas
- webdriver manager
- openpyxl

In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import urllib3
import re
from bs4 import BeautifulSoup
import csv
http=urllib3.PoolManager()
import requests
import json

# Scraping all the links of hackathons

Set ScrollNumber variable in code to any number to gather more links of hackathons hosted on this site

In [13]:
driver = webdriver.Chrome(ChromeDriverManager().install())

#The website that consists of all the hackathons and their links
driver.get("https://unstop.com/hackathons?filters=,all,all,all&types=teamsize,payment,eligible,oppstatus")

#for just testing purposes the number of scrolls are set to 3, we can increase it any number depending on the 
# hackathons we want to extract contact information for.
ScrollNumber = 100
for i in range(1,ScrollNumber):
    driver.execute_script("window.scrollBy(0,document.body.scrollHeight)")
    time.sleep(5)

# DS.html has the html code for the infinite scrolling page    
file = open('DS.html', 'w')
file.write(driver.page_source)
file.close()

driver.close()

[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 96.0.4664
INFO:WDM:Current google-chrome version is 96.0.4664
[WDM] - Get LATEST chromedriver version for 96.0.4664 google-chrome
INFO:WDM:Get LATEST chromedriver version for 96.0.4664 google-chrome
[WDM] - Driver [/home/rohit/.wdm/drivers/chromedriver/linux64/96.0.4664.45/chromedriver] found in cache
INFO:WDM:Driver [/home/rohit/.wdm/drivers/chromedriver/linux64/96.0.4664.45/chromedriver] found in cache


## Scrape the HTML code and extract links for all the hackathons and output to excel

In [20]:
def scrape(file_name='DS.html'):
    contact_link_list = []
    contacts_json_list = []
    
    data = open(file_name,'r')
    soup = BeautifulSoup(data, 'html.parser')
    # find according to regex (listing order-\d id-[0-9]+ full_banner ng-star-inserted)
    for link in soup.find_all('a', {'class': re.compile(r"listing order-[A-Za-z0-9]+ id-[0-9]+", re.IGNORECASE)}):
        print('https://unstop.com'+link.get('href'))
        contact_link, contacts_json = extract_contacts('https://unstop.com'+link.get('href'))
    
        
        contact_link_list.append(contact_link)
        contacts_json_list.append(contacts_json)
    return contact_link_list, contacts_json_list

# Using the API to extract contacts:
def extract_contacts(link):
    
    contact_link = ''
    contacts_json = ''
    non_contact_link = ''
    
    
    last_bit= link.split('/')[-1].split('-')[-1]
    response_API = requests.get('https://unstop.com/api/public/competition/'+last_bit)
    if last_bit.isdigit():
        data = response_API.text
        parse_json = json.loads(data)
        if 'data' in parse_json and 'competition' in parse_json['data']:
            if 'contacts' in parse_json['data']['competition']:
                contact_link = link
                contacts_json = parse_json['data']['competition']['contacts']
                return contact_link, contacts_json
            else:
                non_contact_link = link
                contacts_json = ''
                return non_contact_link, contacts_json
    else:
        return '',''

contact_link_list, contacts_json_list = scrape('DS.html')
length = [len(i) for i in contacts_json_list]
df = pd.DataFrame({'hackathon links': contact_link_list, 'Contact_information': contacts_json_list, 'length':length})
contacts_df = df[df['length']!=0][['hackathon links','Contact_information']]
non_contacts_df = df[df['length']==0][['hackathon links','Contact_information']]

contacts_df.to_excel('contacts.xlsx', sheet_name='sheet1', index=False)
non_contacts_df.to_excel('non_contacts.xlsx', sheet_name='sheet1', index=False)


https://unstop.com/hackathon/wecode-coding-contest-21-graphic-era-hill-university-gehu-bhimtal-490832
https://unstop.com/hackathon/queen-city-hacks-queen-city-hacks-490822
https://unstop.com/hackathon/rock-hacks-rock-hacks-490603
https://unstop.com/hackathon/gdscs-hackin-winters-jaypee-institute-of-information-technology-jiit-noida-489252
https://unstop.com/hackathon/line-following-robot-enyugma-22-indian-institute-of-information-technology-iiit-bhagalpur-489171
https://unstop.com/hackathon/bit-by-bit-enyugma-22-indian-institute-of-information-technology-iiit-bhagalpur-486498
https://unstop.com/hackathon/codeloop-enyugma-22-indian-institute-of-information-technology-iiit-bhagalpur-482617
https://unstop.com/hackathon/inspirathon-20-netaji-subhas-university-of-technology-nsut-delhi-484390
https://unstop.com/hackathon/crypton-2022-the-northcap-university-ncu-gurgaon-486704
https://unstop.com/hackathon/codeverse-indian-institute-of-technology-iit-bhubaneswar-483697
https://unstop.com/hacka

https://unstop.com/hackathon/bugsandbyte-riskcovry-435723
https://unstop.com/hackathon/muj-hacks-70-elicit22-manipal-university-mu-jaipur-439018
https://unstop.com/hackathon/ycp-hacks-york-college-of-pennsylvania-438753
https://unstop.com/hackathon/hybrid-hack-aaruush-srm-institute-of-science-and-technology-srmist-kattankulathur-chennai-srm-institute-of-sci-438278
https://unstop.com/hackathon/hinterland-hack-2022-founders-foundation-433048
https://unstop.com/hackathon/quant-a-maze-2022-nitte-meenakshi-institute-of-technology-nmit-yelahanka-432616
https://unstop.com/hackathon/wecode-coding-contest-17-graphic-era-hill-university-gehu-bhimtal-435585
https://unstop.com/hackathon/ideate2sustain-indira-gandhi-delhi-technical-university-for-women-igdtuw-delhi-433574
https://unstop.com/hackathon/hawk-the-log-retail-dataset-competition-interface-2022-birla-institute-of-technology-science-bits-pilani-436615
https://unstop.com/hackathon/newhacks-2022-university-of-toronto-436543
https://unstop.co

https://unstop.com/hackathon/hack-with-ai-10-bv-raju-institute-of-technology-tuljaraopet-telangana-363369
https://unstop.com/hackathon/code-fin-20-a-coding-quiz-missioned-348292
https://unstop.com/hackathon/codexthon-malaviya-national-institute-of-technology-mnit-jaipur-360380
https://unstop.com/hackathon/code-with-globalshala-30-globalshala-362115
https://unstop.com/hackathon/code-fin-20-a-coding-quiz-light-school-app-361753
https://unstop.com/hackathon/code-plus-plus-vellore-institute-of-technology-vit-vellore-358587
https://unstop.com/hackathon/hackathon-jumpstart-publicis-sapient-359785
https://unstop.com/hackathon/tally-codebrewers-tally-solutions-343510
https://unstop.com/hackathon/house-of-code-senior-data-engineer-house-of-code-code-to-possibilities-lowes-companies-inc-360189
https://unstop.com/hackathon/ai-hack-22-rns-institute-of-technology-rnsit-bengaluru-359266
https://unstop.com/hackathon/house-of-code-data-engineer-house-of-code-code-to-possibilities-lowes-companies-inc-3

https://unstop.com/hackathon/fin-o-hack-fortune-22-delhi-technological-university-dtu-new-delhi-281639
https://unstop.com/hackathon/logicape-tryst-indian-institute-of-technology-iit-delhi-281409
https://unstop.com/hackathon/bugsmash-tryst-indian-institute-of-technology-iit-delhi-281381
https://unstop.com/hackathon/game-jam-tryst-indian-institute-of-technology-iit-delhi-281262
https://unstop.com/hackathon/data-hackathon-sri-venkateswara-college-svc-delhi-university-delhi-281200
https://unstop.com/hackathon/maze-race-technunctus22-indian-institute-of-technology-iit-jammu-280751
https://unstop.com/hackathon/name-it-technunctus22-indian-institute-of-technology-iit-jammu-280733
https://unstop.com/hackathon/simhack-apogee-bits-pilani-the-encrypted-dimension-birla-institute-of-technology-science-bits-pilani-276052
https://unstop.com/hackathon/ideathon-avathharanae-graduate-school-of-management-studies-gsms-280651
https://unstop.com/hackathon/code-it-out-apogee-bits-pilani-the-encrypted-dimens

https://unstop.com/hackathon/hackaverse-spectra-the-fest-sardar-patel-college-of-engineering-spce-mumbai-268534
https://unstop.com/hackathon/pay-hack-global-entrepreneurship-summit-2022-innovation-for-inclusion-ges-2022-indian-institute-of-technology--268698
https://unstop.com/hackathon/code-character-bytehoc-pragyan-seshasayee-institute-of-technologysit-tiruchirappalli-268488
https://unstop.com/hackathon/global-grand-coding-contest-newton-schoolns-bengaluru-268519
https://unstop.com/hackathon/css-champs-technozarre22-srm-institute-of-science-and-technology-vadapalani-campus-267846
https://unstop.com/hackathon/mindsweeper-north-india-zonals-cognizance-2022-indian-institute-of-technology-iit-roorkee-267490
https://unstop.com/hackathon/innovaccers-hackercamp22-powered-by-microsoft-innovaccers-hackercamp22-powered-by-microsoft-innovaccer-266149
https://unstop.com/hackathon/innovaccers-hackercamp22-powered-by-microsoft-innovaccers-hackercamp22-powered-by-microsoft-innovaccer-267422
https:/

https://unstop.com/hackathon/hack-the-game-cepheus22-annual-technical-fest-iit-goa-indian-institute-of-technology-iit-goa-252532
https://unstop.com/hackathon/source-code-kascade-bihar-jharkhand-kshitij-indian-institute-of-technology-iit-kharagpur-251724
https://unstop.com/hackathon/kode-a-thon-college-of-engineering-ceg-guindy-251862
https://unstop.com/hackathon/dframe-the-uiux-design-hackathon-udgam-2022-indian-institute-of-technology-iit-guwahati-251886
https://unstop.com/hackathon/cuhackit-2022-college-of-engineering-computing-applied-sciences-clemson-university-251578
https://unstop.com/hackathon/designathon-build-exun-2021-22-delhi-public-school-r-k-puram-251138
https://unstop.com/hackathon/unreality-build-exun-2021-22-delhi-public-school-r-k-puram-251131
https://unstop.com/hackathon/build-hackathon-exun-2021-22-delhi-public-school-r-k-puram-251139
https://unstop.com/hackathon/source-code-kascade-tamil-nadu-indian-institute-of-technology-iit-kharagpur-251549
https://unstop.com/hac

https://unstop.com/hackathon/internathon-50-harbour-x-232733
https://unstop.com/hackathon/tiger-hacks-university-of-missouri-columbia-232370
https://unstop.com/hackathon/spectra-st-josephs-college-of-engineering-chennai-231507
https://unstop.com/hackathon/ecovision-centre-for-innovation-and-incubation-nitw-230963
https://unstop.com/hackathon/hackuiet-university-institute-of-engineering-and-technology-uiet-chandigarh-231364
https://unstop.com/hackathon/interizon-hackathon-2021-interizon-hacks-230726
https://unstop.com/hackathon/space-invaders-innerve-2021-indira-gandhi-delhi-technical-university-for-women-igdtuw-delhi-228358
https://unstop.com/hackathon/ideathon-for-the-entrepreneur-in-you-other-organization-not-listed-here-224308
https://unstop.com/hackathon/babylon-project-v2-world-blockchain-hackathon-164223
https://unstop.com/creative-cultural-event/web-o-code-bvest-2021-bharati-vidyapeeths-college-of-engineering-bvcoe-new-delhi-226599
https://unstop.com/hackathon/what-the-haick-inn

https://unstop.com/hackathon/robohack-10-flyhawk-robospace-pvt-ltd-195307
https://unstop.com/hackathon/ideathon-envisage21-vellore-institute-of-technology-vit-chennai-195095
https://unstop.com/hackathon/auto-potency-hackodisha-national-institute-of-technology-nit-rourkela-194981
https://unstop.com/hackathon/replication-hackodisha-national-institute-of-technology-nit-rourkela-194970
https://unstop.com/hackathon/hackrice-11-rice-university-194514
https://unstop.com/hackathon/beyond-analysis-a-data-science-hackathon-techniche-indian-institute-of-technology-iit-guwahati-194331
https://unstop.com/hackathon/innovators-hub-dr-vishwanath-karad-mit-world-peace-university-mit-wpu-pune-194089
https://unstop.com/hackathon/code-crush-delhi-technological-university-dtu-new-delhi-194017
https://unstop.com/hackathon/wordpress-task-completion-startup-shiksha-193735
https://unstop.com/hackathon/coderatri-20-vellore-institute-of-technology-vit-chennai-193725
https://unstop.com/hackathon/futuristfest-2021

In [19]:
last_bit = '45678'
test = 'abcd'
test.isdigit()

False

In [21]:
len(contact_link_list), len(contacts_json_list), len(length)

(660, 660, 660)

micromouse was the last hackathon for 50 scrolls, lets check what is the last hackathon for 100 scrolls. It will give us some idea whether we are properly squeezing out all the information from the site.

[a-zA-Z]+-bg opp-type t ng-star-inserted # regex for the class for extracting text like hackathon or cultural event etc
and tag to search under is h4

double-wrap ng-star-inserted # search for this class when extracting college name under the tag h3

In [ ]:
## Using the api method instead of scrapping method(so the code below is not useful for now)

# ## Trying to figure out how to extract contact information:

# ### Stuck here: [created a forum link](https://forum.webscraper.io/t/extract-contact-information-from-website-but-stuck/10543)

# import os
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# import time
# from webdriver_manager.chrome import ChromeDriverManager
# import pandas as pd

# driver = webdriver.Chrome(ChromeDriverManager().install())

# driver.get("https://unstop.com/hackathon/weave-the-web-powered-by-edot-solutions-techyon-2022-padre-conceicao-college-of-engineering-salcete-goa-462438")
# stages = driver.find_elements_by_xpath("/html/body/div[1]/main/app-public-competition/div/div[2]/div[2]/div/div[1]/div[3]/div/app-dates-and-contacts/section/div/div[2]/ul")
# print(stages)


# ## For contacts extract all of the content under the class "contact ng-star-inserted"